In [1]:
import json
import os.path
import csv
import pandas as pd
import pylab as pl
from datetime import datetime as dt

In [2]:
NUM_USERS = 400
user_data = {}
call_logs = {}
contact_lists = {}
sms_logs = {}
user_status = {}

In [3]:
#initialize call logs, contact lists and sms logs to empty list
for i in range(1, NUM_USERS+1):
    call_logs[i] = []
    contact_lists[i] = []
    sms_logs[i] = []

#### Get the status of each user and save it as a dictionary

In [4]:
with open("../data/user_logs/user_status.csv", 'rb') as csvfile:
    has_header = csv.Sniffer().has_header(csvfile.read(1024))
    csvfile.seek(0)  # Rewind
    reader = csv.reader(csvfile)
    if has_header:
        next(reader)  # Skip header row
    for row in reader:
        user_id = int(row[0])
        user_status[user_id] = []
        user_status[user_id].append(row[1])  # disbursement time
        user_status[user_id].append(row[2])  # status

#### Get the call log, contact list, and sms log for each user and store them in their corresponding dictionaries.  While we're at it, get the number of devices

In [5]:
user_data["num_devices"] = []
for user_id in range(1, NUM_USERS+1):
    path = './../data/user_logs/user-' + str(user_id) + '/'
    devices = [name for name in os.listdir(path)]
    for device in devices:
        call_log_path = path + device + '/collated_call_log.txt'
        contact_list_path = path + device + '/collated_contact_list.txt'
        sms_log_path = path + device + '/collated_sms_log.txt'
        try:
            with open(call_log_path) as call_log_file:
                call_logs[user_id].extend(json.load(call_log_file))
        except IOError:
            print "Error loading file"
        try:
            with open(contact_list_path) as contact_list_file:
                contact_lists[user_id].extend(json.load(contact_list_file))
        except IOError:
            print "Error loading file"
        try:
            with open(sms_log_path) as sms_log_file:
                sms_logs[user_id].extend(json.load(sms_log_file))
        except IOError:
            print "Error loading file"
            
    #print str(user_id) + ":",str(len(devices)), "devices"        
    user_data["num_devices"].append(len(devices))

Error loading file
Error loading file
Error loading file
Error loading file
Error loading file
Error loading file
Error loading file
Error loading file


#### Get some user statistics: number of calls, number of sms messages, number of contacts, number of devices

In [6]:
user_data["num_calls"] = []
user_data["num_contacts"] = []
user_data["num_sms"] = []
user_data["user_id"] = []
for user_id in range(1, NUM_USERS+1):
    user_data["user_id"].append(user_id)
    user_data["num_calls"].append(len(call_logs[user_id]))
    user_data["num_contacts"].append(len(contact_lists[user_id]))
    user_data["num_sms"].append(len(sms_logs[user_id]))

In [7]:
user_data_df = pd.DataFrame(data=user_data)
user_data_df.head()

,num_calls,num_contacts,num_devices,num_sms,user_id
0,182,246,1,133,1
1,1423,88,1,3773,2
2,442,200,1,125,3
3,1775,711,1,10522,4
4,2356,128,1,1654,5


#### Let's see what this dataset looks like

In [8]:
user_data_df[['num_calls','num_contacts','num_sms','num_devices']].describe()

,num_calls,num_contacts,num_sms,num_devices
count,400.000000,400.000000,400.000000,400.000000
mean,1588.225000,573.585000,3336.160000,1.255000
std,1704.651581,1027.860431,6028.870615,0.718516
min,4.000000,14.000000,4.000000,1.000000
25%,500.000000,209.000000,410.250000,1.000000
50%,1000.000000,344.500000,1571.500000,1.000000
75%,2137.250000,594.750000,4443.500000,1.000000
max,11821.000000,13850.000000,86546.000000,7.000000


In [9]:
user_data_df[['num_calls','num_contacts','num_sms','num_devices']].hist()
pl.show()

#### Let's see how many previous loans the users have been awarded. The intuition is that if a user has been awarded sevaral loans in the past, they will probably be more likely to repay this one. 

#### We will look at the user's sms messages to find messages that were sent by "Branch-Co" and start with "Your Branch loan of...", as this is what awarded loan sms messages look like.

In [22]:
def get_num_prev_loans(user_id, disbursement_date):
    loan_date = dt.strptime(disbursement_date,"%Y-%m-%dT%H:%M:%S.%fZ")
    count_prev_loans = 0
    smss = sms_logs[user_id]
    for sms in smss:
        if "sms_address" in sms and "message_body" in sms and "datetime" in sms:
            if sms["sms_address"] == "Branch-Co" and sms["message_body"].startswith("Your Branch loan of"):
                sms_time = dt.fromtimestamp(float(sms["datetime"])/1000)

                if sms_time < loan_date:
                    count_prev_loans += 1
                    print "disbursement_date:", disbursement_date
                    print "->", sms["message_body"]
                    print ">", sms_time
        #else:
            #print "~~~ ERROR ~~~"
    return count_prev_loans

In [23]:
for user in user_status:
    print user, get_num_prev_loans(user, user_status[user][0])

1 disbursement_date: 2017-05-29T00:00:00.000Z
-> Your Branch loan of Ksh 1,000 has been sent! Your first repayment is due in 7 days to Paybill: 998608, Account: +254728798421. If you like Branch, we would appreciate a 5-star review in Google Play: http://bit.ly/branch_co 
> 2017-05-28 21:25:10.678000
1
2 disbursement_date: 2017-05-30T00:00:00.000Z
-> Your Branch loan of Ksh 1,000 has been sent! Your first repayment is due in 7 days to Paybill: 998608, Account: +254795652875. If you like Branch, we would appreciate a 5-star review in Google Play: http://bit.ly/branch_co 
> 2017-05-29 23:51:41.602000
1
3 0
4 0
5 0
6 0
7 disbursement_date: 2017-05-30T00:00:00.000Z
-> Your Branch loan of Ksh 1,000 has been sent! Your first repayment is due in 7 days to Paybill: 998608, Account: +254796256956. If you like Branch, we would appreciate a 5-star review in Google Play: http://bit.ly/branch_co 
> 2017-05-29 23:50:44.915000
1
8 0
9 0
10 0
11 disbursement_date: 2017-05-28T00:00:00.000Z
-> Your Bran

#### Some users have been awarded multiple loans (e.g. user 9 was awarded 12 loans), however, when we look at the time of these award sms messages, we see that they all happened before the current loan (i.e. the loan in the 'user_status.csv' file)
#### Looking at the results above, there are some users that seem to have one previous loan. However, taking a closer look at them, we can see that the award messages were sent late at night the night before the disbursement date.
#### This means that we are considering the users' first loans. This in turn means that not only can we not use previous loan information, but we also should disregard all messages (and calls) that happened after the disbursement date, which is in the future.